In [1]:
from IPython.display import Image
import ee

ee.Initialize()

In [2]:
# Input data
area_of_interest = ee.FeatureCollection('ft:1ihcmnTQF2dUYTKXOIYYwYlJzFLSpO7zsIxg0Yqd5')
mexico_landcover_2010_landsat = ee.Image("users/renekope/MEX_LC_2010_Landsat_v43").clip(area_of_interest)
landsat8_collection = ee.ImageCollection('LANDSAT/LC8_L1T_TOA').filterDate('2016-01-01', '2016-04-19').min()
landsat8_collection = landsat8_collection.slice(0,9)

In [3]:
# Functions to derive vegetation indices and other raster operations
def NDVI(image):
    return image.normalizedDifference(['B5', 'B4'])

def SAM(image):
    band1 = image.select("B1")
    bandn = image.select("B2","B3","B4","B5","B6","B7","B8","B9");
    maxObjSize = 256;
    b = band1.divide(bandn);
    spectralAngleMap = b.atan();
    spectralAngleMap_sin = spectralAngleMap.sin();
    spectralAngleMap_cos = spectralAngleMap.cos();
    sum_cos = spectralAngleMap_cos.reduce(ee.call("Reducer.sum"));
    sum_sin = spectralAngleMap_sin.reduce(ee.call("Reducer.sum"));
    return ee.Image.cat(sum_sin, sum_cos, spectralAngleMap_sin, spectralAngleMap_cos);

#Enhanced Vegetation Index
def EVI(image):
    # L(Canopy background)
    # C1,C2(Coefficients of aerosol resistance term)
    # GainFactor(Gain or scaling factor)
    gain_factor = ee.Image(2.5);
    coefficient_1 = ee.Image(6);
    coefficient_2 = ee.Image(7.5);
    l = ee.Image(1);
    nir = image.select("B5");
    red = image.select("B4");
    blue = image.select("B2");
    evi = image.expression(
        "Gain_Factor*((NIR-RED)/(NIR+C1*RED-C2*BLUE+L))",
        {
            "Gain_Factor":gain_factor,
            "NIR":nir,
            "RED":red,
            "C1":coefficient_1,
            "C2":coefficient_2,
            "BLUE":blue,
            "L":l
        }
    )
    return evi

#Atmospherically Resistant Vegetation Index
def ARVI(image):
    red = image.select("B4")
    blue = image.select("B2")
    nir = image.select("B5")
    red_square = red.multiply(red)
    arvi = image.expression(
        "NIR - (REDsq - BLUE)/(NIR+(REDsq-BLUE))",{
            "NIR": nir,
            "REDsq": red_square,
            "BLUE": blue
        }
    )
    return arvi

#Leaf Area Index
def LAI(image):
    nir = image.select("B5")
    red = image.select("B4")
    coeff1 = ee.Image(0.0305);
    coeff2 = ee.Image(1.2640);
    lai = image.expression(
        "(((NIR/RED)*COEFF1)+COEFF2)",
        {
            "NIR":nir,
            "RED":red,
            "COEFF1":coeff1,
            "COEFF2":coeff2
        }
    )
    return lai

def tasseled_cap_transformation(image):
    #Tasseled Cap Transformation for Landsat 8 based on the 
    #scientfic work "Derivation of a tasselled cap transformation based on Landsat 8 at-satellite reflectance" 
    #by M.Baigab, L.Zhang, T.Shuai & Q.Tong (2014). The bands of the output image are the brightness index, 
    #greenness index and wetness index.
    b = image.select("B2", "B3", "B4", "B5", "B6", "B7");
    #Coefficients are only for Landsat 8 TOA
    brightness_coefficents= ee.Image([0.3029, 0.2786, 0.4733, 0.5599, 0.508, 0.1872])
    greenness_coefficents= ee.Image([-0.2941, -0.243, -0.5424, 0.7276, 0.0713, -0.1608]);
    wetness_coefficents= ee.Image([0.1511, 0.1973, 0.3283, 0.3407, -0.7117, -0.4559]);
    fourth_coefficents= ee.Image([-0.8239, 0.0849, 0.4396, -0.058, 0.2013, -0.2773]);
    fifth_coefficents= ee.Image([-0.3294, 0.0557, 0.1056, 0.1855, -0.4349, 0.8085]);
    sixth_coefficents= ee.Image([0.1079, -0.9023, 0.4119, 0.0575, -0.0259, 0.0252]);
    
    #Calculate tasseled cap transformation
    brightness = image.expression(
        '(B * BRIGHTNESS)',
        {
            'B':b,
            'BRIGHTNESS': brightness_coefficents
        })
    greenness = image.expression(
        '(B * GREENNESS)',
        {
            'B':b,
            'GREENNESS': greenness_coefficents
        })
    wetness = image.expression(
        '(B * WETNESS)',
        {
            'B':b,
            'WETNESS': wetness_coefficents
        })
    fourth = image.expression(
        '(B * FOURTH)',
        {
            'B':b,
            'FOURTH': fourth_coefficents
        })
    fifth = image.expression(
        '(B * FIFTH)',
        {
            'B':b,
            'FIFTH': fifth_coefficents
        })
    sixth = image.expression(
        '(B * SIXTH)',
        {
            'B':b,
            'SIXTH': sixth_coefficents
        })
    bright = brightness.reduce(ee.call("Reducer.sum"));
    green = greenness.reduce(ee.call("Reducer.sum"));
    wet = wetness.reduce(ee.call("Reducer.sum"));
    four = fourth.reduce(ee.call("Reducer.sum"));
    five = fifth.reduce(ee.call("Reducer.sum"));
    six = sixth.reduce(ee.call("Reducer.sum"));
    tasseled_cap = ee.Image(bright).addBands(green).addBands(wet).addBands(four).addBands(five).addBands(six)
    return tasseled_cap.rename('brightness','greenness','wetness','fourth','fifth','sixth')

In [4]:
tct = tasseled_cap_transformation(landsat8_collection)

In [5]:
tct.bandNames().getInfo()

['brightness', 'greenness', 'wetness', 'fourth', 'fifth', 'sixth']

In [6]:
#Display thumbnail of classification output
image = tct.clip(area_of_interest)
theGeom = area_of_interest.geometry().getInfo()['coordinates']
thumbnail = image.getThumbUrl({'min':-1,'max':1,'size':'800','region':theGeom})
Image(url=thumbnail)

In [7]:
#Derive indices, spectral angles and build an image stack
ndvi = NDVI(landsat8_collection)
sam = SAM(landsat8_collection)
evi = EVI(landsat8_collection)
arvi = ARVI(landsat8_collection)
lai = LAI(landsat8_collection)
spectral_indices_stack = ee.Image(ndvi).addBands(lai).addBands(sam).addBands(arvi).addBands(evi)

In [8]:
#Define classification function
def classification(raster_input, training_dataset,number_of_training_points, region, classification_algorithm):
    bands = raster_input.bandNames()
    points = ee.FeatureCollection.randomPoints(region, number_of_training_points, number_of_training_points, 1)
    training = training_dataset.addBands(raster_input).reduceToVectors(
        reducer="mean",
        geometry=points,
        geometryType="centroid",
        scale=30, 
        crs="EPSG:4326"
    )

    classifier = training.trainClassifier(
        property_list=bands,
        class_property="label",
        classifier_name=classification_algorithm
    )
    # Apply the classifier to the original composite.
    out = raster_input.classify(classifier)
    return out

In [9]:
#Execute classification function
output = classification(spectral_indices_stack, mexico_landcover_2010_landsat, 1000, area_of_interest, 'Cart')

In [10]:
#Display thumbnail of classification output
image = output.clip(area_of_interest)
theGeom = area_of_interest.geometry().getInfo()['coordinates']
thumbnail = image.getThumbUrl({'min':0,'max':211,'size':'800','region':theGeom})
Image(url=thumbnail)


New test from api example

In [7]:
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318')

vizParams = {'bands': ['B5', 'B4', 'B3'],
             'min': 0,
             'max': 0.5,
             'gamma': [0.95, 1.1, 1]
}